In [ ]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Configurações iniciais =======================================================
# Carregar dados (substitua por seu dataset real)
# df = pd.read_excel(LOCAL_OUTPUT_PATH)
# Simulando dados para exemplo:
dates = pd.date_range('2016-01-01', '2024-12-31', freq='M')
df = pd.DataFrame({
    'date': dates,
    'periodo': np.arange(len(dates)),  # Variável temporal numérica
    'valores': np.random.normal(loc=100, scale=20, size=len(dates))  # Variável dependente simulada
})

# Definir parâmetros temporais críticos =========================================
PANDEMIA_START_DATE = "2020-01-01"
PANDEMIA_END_DATE = "2020-08-01"
SECONDLOCKDOWN_START_DATE = "2021-04-01"
SECONDLOCKDOWN_END_DATE = '2021-05-01'
POS_PANDEMIA_START_DATE = "2020-01-01"

# Criar variáveis dummy críticas ================================================
# Ponto 1: Criação correta da trend pós-pandemia para evitar sobreposição
df['Pulse_Pandemia'] = np.where(
    (df['date'] >= PANDEMIA_START_DATE) & (df['date'] <= PANDEMIA_END_DATE), 1, 0)

df['Second_Lockdown'] = np.where(
    (df['date'] >= SECONDLOCKDOWN_START_DATE) & (df['date'] <= SECONDLOCKDOWN_END_DATE), 1, 0)

# Trend_PandemiaePos: Variável incremental a partir de POS_PANDEMIA_START_DATE
# Ponto 2: Garantir que começa em 1 e não há overlap com 'periodo'
pandemic_start_idx = df[df['date'] == POS_PANDEMIA_START_DATE].index[0]
df['Trend_PandemiaePos'] = np.where(
    df['date'] >= POS_PANDEMIA_START_DATE,
    df.index - pandemic_start_idx + 1,  # +1 para começar em 1
    0
)

# Modelo 1: Especificação original com tendência contínua =======================
# Ponto 3: Fórmula que combina tendência base + efeitos pandêmicos
modelo_original = smf.ols(
    'valores ~ periodo + Pulse_Pandemia + Second_Lockdown + Trend_PandemiaePos',
    data=df
).fit()

# Modelo 2: Especificação segmentada para comparação ============================
# Ponto 4: Criação correta das variáveis segmentadas para evitar multicolinearidade
df['Pre_Pandemia_trend'] = np.where(df['date'] < PANDEMIA_START_DATE, df['periodo'], 0)
df['Pos_Pandemia_trend'] = np.where(df['date'] >= PANDEMIA_START_DATE, df['periodo'], 0)

modelo_segmentado = smf.ols(
    'valores ~ Pre_Pandemia_trend + Pos_Pandemia_trend + Pulse_Pandemia + Second_Lockdown',
    data=df
).fit()

# Análise de resultados ========================================================
# Ponto 5: Comparação crítica entre os modelos
print("\n=== Modelo Original (tendência contínua + ajuste pós) ===")
print(modelo_original.summary())

print("\n=== Modelo Segmentado (tendências separadas) ===")
print(modelo_segmentado.summary())

# Ponto 6: Verificação da equivalência matemática dos modelos
slope_pre_original = modelo_original.params['periodo']
slope_pos_original = modelo_original.params['periodo'] + modelo_original.params['Trend_PandemiaePos']

slope_pre_segmentado = modelo_segmentado.params['Pre_Pandemia_trend']
slope_pos_segmentado = modelo_segmentado.params['Pos_Pandemia_trend']

print(f"\nComparação de slopes:")
print(f"Original -> Pré: {slope_pre_original:.2f}, Pós: {slope_pos_original:.2f}")
print(f"Segmentado -> Pré: {slope_pre_segmentado:.2f}, Pós: {slope_pos_segmentado:.2f}")

# Diagnóstico de multicolinearidade ============================================
# Ponto 7: Verificação crítica para os modelos
def calculate_vif(model):
    vif_data = pd.DataFrame()
    vif_data["Variável"] = model.model.exog_names
    vif_data["VIF"] = [variance_inflation_factor(model.model.exog, i) 
                      for i in range(model.model.exog.shape[1])]
    return vif_data

print("\nVIF Modelo Original:")
print(calculate_vif(modelo_original))

print("\nVIF Modelo Segmentado:")
print(calculate_vif(modelo_segmentado))

# Visualização dos resultados ==================================================
# Ponto 8: Gráfico para verificar consistência das tendências
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(df['date'], df['valores'], 'o', label='Dados Observados', alpha=0.5)

# Previsões dos modelos
df['pred_original'] = modelo_original.predict()
df['pred_segmentado'] = modelo_segmentado.predict()

plt.plot(df['date'], df['pred_original'], 'r-', label='Modelo Original')
plt.plot(df['date'], df['pred_segmentado'], 'g--', label='Modelo Segmentado')

plt.axvline(pd.to_datetime(PANDEMIA_START_DATE), color='gray', linestyle='--', label='Início Pandemia')
plt.title('Comparação dos Modelos')
plt.legend()
plt.show()

# Análise de resíduos ==========================================================
# Ponto 9: Verificação crítica de pressupostos
fig, axes = plt.subplots(1, 2, figsize=(15, 5))
modelo_original.resid.plot(ax=axes[0], title='Resíduos - Modelo Original')
modelo_segmentado.resid.plot(ax=axes[1], title='Resíduos - Modelo Segmentado')
plt.show()

# Notas importantes ============================================================
"""
Pontos Críticos Resolvidos:
1. Definição da Trend_PandemiaePos: Criada como variável incremental separada da tendência base
2. Não-overlap de variáveis: Modelo original mantém 'periodo' contínuo enquanto dummies capturam efeitos específicos
3. Comparabilidade: Slopes pós-pandemia devem ser equivalentes entre modelos (original: base + ajuste vs segmentado: direto)
4. Diagnóstico: Verificação de VIF para garantir que multicolinearidade não distorce resultados
5. Validação Visual: Gráficos e análise de resíduos confirmam consistência dos modelos

Cuidados Adicionais:
- O intercepto representa o valor esperado quando todas as variáveis são zero
- Se 'periodo' não começar em zero, a interpretação do intercepto muda
- Para séries temporais, verificar autocorrelação residual (Durbin-Watson)
"""